<a href="https://colab.research.google.com/github/madhusudanhasbe/Natural_Language_Processing_Lab/blob/main/22070126061_NLP_Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 5

Name: Madhusudan Hasbe\
PRN: 22070126061\
TY AIML A3

Github Link: https://github.com/madhusudanhasbe/Natural_Language_Processing_Lab/blob/main/22070126061_NLP_Assignment_5.ipynb

## Question Answering by Fine-Tuning T5

#### Imports

In [ ]:
!pip install datasets

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW, get_scheduler
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import os
from google.colab import drive
from nltk.translate.bleu_score import sentence_bleu
import nltk

# Download NLTK data for BLEU score calculation
nltk.download('punkt')

In [ ]:
# Define the directory to save the model in Google Drive
save_directory = '/content/drive/MyDrive/T5_finetuned_CoQA'
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

#### Load Dataset and Process Dataset

In [ ]:
# Load the CoQA dataset
coqa = load_dataset('coqa')

In [ ]:
# Custom Dataset Class for CoQA
class CoQADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        question = item['questions'][0]
        context = item['story']
        answer = item['answers']['input_text'][0]  # Take the first answer (CoQA provides multiple answers)

        # Input: question + context
        input_text = f"question: {question}  context: {context} </s>"
        inputs = self.tokenizer(
            input_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt"
        )

        # Target: Answer
        targets = self.tokenizer(
            answer, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt"
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': targets['input_ids'].squeeze(),
        }

In [ ]:
# Initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device)

In [ ]:
# Prepare data for training
train_data = coqa['train']
train_dataset = CoQADataset(train_data, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [ ]:
# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = len(train_loader) * 3  # 3 epochs
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# Initialize variables for tracking the best model
best_loss = float('inf')

#### Training Function

In [ ]:
# Training loop with tqdm progress bar and model saving
model.train()
for epoch in range(3):  # Adjust number of epochs as needed
    epoch_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}", leave=True)

    for batch in progress_bar:
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        epoch_loss += loss.item()
        progress_bar.set_postfix({"Loss": loss.item()})

    avg_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch + 1} finished. Average Loss: {avg_loss}")

    # Save the model if it's the best so far
    if avg_loss < best_loss:
        best_loss = avg_loss
        model.save_pretrained(save_directory)
        tokenizer.save_pretrained(save_directory)
        print(f"Best model saved with avg_loss: {avg_loss}")

print(f"Training complete. Best model saved at: {save_directory}")

#### BLEU Score

In [ ]:
# Function to generate an answer from the model for a single input
def generate_answer(model, tokenizer, input_text, device, max_length=512):
    # Tokenize input text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=max_length, truncation=True).to(device)

    # Generate answer
    with torch.no_grad():
        outputs = model.generate(inputs['input_ids'], max_length=max_length, num_beams=2, early_stopping=True)

    # Decode the answer from token IDs to text
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Function to calculate BLEU score for a single prediction and reference
def calculate_bleu(prediction, reference):
    if isinstance(reference, list):
      reference = reference[0]
    return sentence_bleu([reference.split()], prediction.split())

# Example usage: Test on a single question + context example
def test_single_example(model, tokenizer, device, question, context, reference_answer):
    if isinstance(question, list):
        question = question[0]
    if isinstance(context, list):
        context = context[0]
    if isinstance(reference_answer, list):
        reference_answer = reference_answer[0]
    # Prepare input: question + context
    input_text = f"question: {question}  context: {context}"

    # Generate the prediction
    predicted_answer = generate_answer(model, tokenizer, input_text, device)

    # Calculate BLEU score
    bleu_score = calculate_bleu(predicted_answer, reference_answer)

    # Print prediction and BLEU score
    print(f"Predicted Answer: {predicted_answer}")
    print(f"Reference Answer: {reference_answer}")
    print(f"BLEU Score: {bleu_score:.4f}")

    return bleu_score

In [ ]:
question = coqa['train']['questions'][7]
context = coqa['train']['story']
reference_answer = coqa['train']['answers'][0]['input_text'][7]

bleu_score = test_single_example(model, tokenizer, device, question, context, reference_answer)

## Question Answering by Fine-Tuning DistilBERT

#### Imports

In [ ]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, AdamW
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
import nltk

# Download NLTK data for BLEU score calculation
nltk.download('punkt')

# Ignore Warnings
import logging
logging.disable(logging.WARNING)

#### Load Dataset and Process Dataset

In [ ]:
# Load CoQA dataset
def load_coqa_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data['data']

# Custom dataset class
class CoQADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        context = item['story']
        question = item['questions'][0]['input_text']
        answer = item['answers'][0]['input_text']

        # Tokenize the input
        inputs = self.tokenizer.encode_plus(
            question,
            context,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Find the start and end positions of the answer in the tokenized input
        input_ids = inputs['input_ids'][0]
        answer_tokens = self.tokenizer.encode(answer, add_special_tokens=False)
        start_position = None
        end_position = None

        for i in range(len(input_ids) - len(answer_tokens) + 1):
            if input_ids[i:i+len(answer_tokens)].tolist() == answer_tokens:
                start_position = i
                end_position = i + len(answer_tokens) - 1
                break

        # If the answer is not found, use the CLS token position as a default
        if start_position is None:
            start_position = 0
            end_position = 0

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'start_positions': torch.tensor(start_position),
            'end_positions': torch.tensor(end_position),
            'answer': answer
        }

In [ ]:
data = load_coqa_data('/content/drive/MyDrive/coqa-train-v1.0.json')
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

In [ ]:
# Initialize tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Prepare datasets and dataloaders
train_dataset = CoQADataset(train_data, tokenizer)
val_dataset = CoQADataset(val_data, tokenizer)
test_dataset = CoQADataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

#### Training Function

In [ ]:
# Training function
def train(model, train_loader, optimizer, device):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc="Training")
    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        progress_bar.set_postfix({'loss': loss.item()})
    return total_loss / len(train_loader)

In [ ]:
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

# Set device and move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

#### Validation and Testing Function

In [ ]:
# Validation function
def validate(model, val_loader, device):
    model.eval()
    total_loss = 0
    progress_bar = tqdm(val_loader, desc="Validating")
    with torch.no_grad():
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_positions = batch['start_positions'].to(device)
            end_positions = batch['end_positions'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
            loss = outputs.loss
            total_loss += loss.item()
            progress_bar.set_postfix({'loss': loss.item()})
    return total_loss / len(val_loader)

In [ ]:
# Test function
def test(model, test_loader, tokenizer, device):
    model.eval()
    all_predictions = []
    all_answers = []
    progress_bar = tqdm(test_loader, desc="Testing")
    with torch.no_grad():
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            answers = batch['answer']
            outputs = model(input_ids, attention_mask=attention_mask)
            start_scores = outputs.start_logits
            end_scores = outputs.end_logits
            for i in range(input_ids.shape[0]):
                start_index = torch.argmax(start_scores[i])
                end_index = torch.argmax(end_scores[i])
                prediction = tokenizer.decode(input_ids[i][start_index:end_index+1])
                all_predictions.append(prediction)
                all_answers.append(answers[i])
    bleu_score = calculate_bleu(all_predictions, all_answers)
    return bleu_score

#### Training

In [ ]:
import os

num_epochs = 3
best_loss = float('inf')
save_path = '/content/drive/My Drive/distilbert_qa_model.pth'

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_loss = train(model, train_loader, optimizer, device)
    val_loss = validate(model, val_loader, device)
    print(f"Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), save_path)
        print("Model saved to Google Drive!")
    else:
        print("Validation Loss Increased. Model Not Saved.")
    print("*" * 50)

#### BLEU Score

In [ ]:
from nltk.translate.bleu_score import SmoothingFunction

# Calculate BLEU score with smoothing
def calculate_bleu(predictions, references):
    smoothie = SmoothingFunction().method4
    bleu_scores = []
    for pred, ref in zip(predictions, references):
        bleu_scores.append(sentence_bleu([ref.split()], pred.split(), smoothing_function=smoothie))
    return sum(bleu_scores) / len(bleu_scores)

In [ ]:
# Test the model
bleu_score = test(model, test_loader, tokenizer, device)
print(f"\nBLEU Score: {bleu_score:.4f}")

In [ ]:
# Create a simple QA bot
def qa_bot(context, question):
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt')
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits

    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)
    answer = tokenizer.decode(input_ids[0][start_index:end_index+1])
    return answer

In [ ]:
test_data[7]['story']

In [ ]:
# Example usage of the QA bot
context = test_data[10]['story']
question = "When is the start of the Julian year?"
answer = qa_bot(context, question)
print(f"Question: {question}")
print(f"Answer: {answer}")

## Question Answering by Fine-Tuning GPT2

#### Imports

In [ ]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
import nltk

nltk.download('punkt')

import logging
logging.disable(logging.WARNING)

#### Load Dataset and Process Dataset

In [ ]:
def load_coqa_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data['data']

class CoQADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        context = item['story']
        question = item['questions'][0]['input_text']
        answer = item['answers'][0]['input_text']

        # Adjusted prompt format
        input_text = f"Context: {context} Question: {question} Answer briefly:"
        target_text = answer

        # Tokenize input and target
        inputs = self.tokenizer.encode_plus(
            input_text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        targets = self.tokenizer.encode_plus(
            target_text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': targets['input_ids'].flatten(),
        }


In [ ]:
data = load_coqa_data('/content/drive/MyDrive/coqa-train-v1.0.json')
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

In [ ]:
# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Set padding token to EOS token

train_dataset = CoQADataset(train_data, tokenizer)
val_dataset = CoQADataset(val_data, tokenizer)
test_dataset = CoQADataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

In [ ]:
# Initialize model
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

#### Training, Validation and Testing Function

In [ ]:
def train(model, train_loader, optimizer, device):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc="Training")

    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({'loss': loss.item()})

    return total_loss / len(train_loader)

In [ ]:
def validate(model, val_loader, device):
    model.eval()
    total_loss = 0
    progress_bar = tqdm(val_loader, desc="Validating")
    with torch.no_grad():
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            progress_bar.set_postfix({'loss': loss.item()})

    return total_loss / len(val_loader)

In [ ]:
def test(model, test_loader, tokenizer, device):
    model.eval()
    all_predictions = []
    all_answers = []
    progress_bar = tqdm(test_loader, desc="Testing")
    with torch.no_grad():
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Generate concise answer
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=30,
                num_beams=3,
                temperature=0.5,
                early_stopping=True
            )
            prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

            all_predictions.append(prediction)

            # Decode true answer for BLEU score calculation
            true_answer = tokenizer.decode(batch['labels'][0], skip_special_tokens=True)
            all_answers.append(true_answer)

    bleu_score = calculate_bleu(all_predictions, all_answers)
    return bleu_score

#### BLEU Score

In [ ]:
def calculate_bleu(predictions, references):
    bleu_scores = [sentence_bleu([ref.split()], pred.split()) for pred, ref in zip(predictions, references)]
    return sum(bleu_scores) / len(bleu_scores)

#### Training

In [ ]:
# Define training loop with evaluation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))
model.to(device)

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training and evaluation function
def train_model(train_loader, val_loader, epochs=3):
    model.train()
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        total_train_loss = 0
        for batch in tqdm(train_loader):
            optimizer.zero_grad()

            # Move data to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_train_loss += loss.item()

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        # Average train loss over the epoch
        avg_train_loss = total_train_loss / len(train_loader)
        print(f"Training loss: {avg_train_loss:.4f}")

        # Validation phase
        model.eval()
        total_val_loss = 0
        bleu_scores = []

        with torch.no_grad():
            for batch in tqdm(val_loader):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                # Forward pass
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                total_val_loss += loss.item()

                # Generate predictions for BLEU score evaluation
                generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=50)
                for i in range(len(generated_ids)):
                    predicted_text = tokenizer.decode(generated_ids[i], skip_special_tokens=True)
                    target_text = tokenizer.decode(labels[i], skip_special_tokens=True)
                    bleu_scores.append(sentence_bleu([target_text.split()], predicted_text.split()))

        # Average validation loss and BLEU score
        avg_val_loss = total_val_loss / len(val_loader)
        avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
        print(f"Validation loss: {avg_val_loss:.4f} - BLEU Score: {avg_bleu_score:.4f}")

train_model(train_loader, val_loader)

In [ ]:
# Training loop
num_epochs = 3
best_loss = float('inf')
save_path = '/content/drive/My Drive/gpt2_qa_model.pth'
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_loss = train(model, train_loader, optimizer, device)
    val_loss = validate(model, val_loader, device)
    print(f"Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), save_path)
        print("Model saved!")
    else:
        print("Validation Loss Increased. Model Not Saved.")
    print("*" * 50)

#### Testing

In [ ]:
# Test the model
bleu_score = test(model, test_loader, tokenizer, device)
print(f"BLEU Score: {bleu_score:.4f}")

In [ ]:
# Create a simple QA bot
def qa_bot(context, question):
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt')
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=20,  # Increase the token limit slightly
            num_beams=20,        # Increase the number of beams for better quality answers
            temperature=1,    # Adjust temperature for more focused responses
            early_stopping=True
        )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [ ]:
test_data[3]['story']

In [ ]:
# Example usage of the QA bot
context = test_data[10]['story']
question = "When is the start of the Julian year?"
answer = qa_bot(context, question)
print(f"Question: {question}")
print(f"Answer: {answer}")

## Gradio Interface

#### Imports

In [ ]:
!pip install gradio

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, DistilBertTokenizer, DistilBertForQuestionAnswering, GPT2Tokenizer, GPT2LMHeadModel
import torch

#### Loading the Trained Models

In [ ]:
# Load T5 from Google Drive
from google.colab import drive
drive.mount('/content/drive')
t5_model_path = '/content/drive/MyDrive/T5_finetuned_CoQA'
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model_path)
t5_model = T5ForConditionalGeneration.from_pretrained(t5_model_path)

# Load DistilBERT from .pth file
distilbert_model_path = '/content/drive/MyDrive/distilbert_qa_model.pth'
distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')
distilbert_model.load_state_dict(torch.load(distilbert_model_path, map_location=torch.device('cpu')))

# Load GPT-2 from .pth file
gpt2_model_path = '/content/drive/MyDrive/gpt2_qa_model.pth'
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')
gpt2_model.load_state_dict(torch.load(gpt2_model_path, map_location=torch.device('cpu')))

#### QA Functions for all Models

In [ ]:
def qa_t5(question, context):
    input_text = f"question: {question} context: {context}"
    input_ids = t5_tokenizer.encode(input_text, return_tensors="pt")
    outputs = t5_model.generate(input_ids)
    answer = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

def qa_distilbert(question, context):
    inputs = distilbert_tokenizer.encode_plus(question, context, return_tensors="pt")
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    outputs = distilbert_model(input_ids=input_ids, attention_mask=attention_mask)
    start_index = torch.argmax(outputs.start_logits)
    end_index = torch.argmax(outputs.end_logits)
    answer = distilbert_tokenizer.decode(input_ids[0][start_index:end_index+1])
    return answer

def qa_gpt2(question, context):
    input_text = f"{question} {context}"
    input_ids = gpt2_tokenizer.encode(input_text, return_tensors="pt")
    outputs = gpt2_model.generate(input_ids, max_length=200)
    answer = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

#### Gradio Interface

In [ ]:
import gradio as gr

def qa_bot(model_name, context, question):
    if model_name == "T5":
        return qa_t5(question, context)
    elif model_name == "DistilBERT":
        return qa_distilbert(question, context)
    elif model_name == "GPT-2":
        return qa_gpt2(question, context)

interface = gr.Interface(
    fn=qa_bot,
    inputs=[
        gr.Radio(choices=["T5", "DistilBERT", "GPT-2"], label="Model"),  # Changed to gr.Radio
        gr.Textbox(lines=10, placeholder="Enter the article (context) here...", label="Context"),  # Changed to gr.Textbox
        gr.Textbox(lines=2, placeholder="Enter the question here...", label="Question")  # Changed to gr.Textbox
    ],
    outputs=gr.Textbox(label="Answer"),  # Changed to gr.Textbox
    title="Question Answering with Multiple Models",
    description="Select a model, enter the context, and ask a question to get the answer."
)

interface.launch()